<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: A_i_rhs_no_gauge_terms.C

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the construction of the right-hand side of the evolution equations of $A_{i}$

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#a_i_rhs_no_gauge_terms__c): **The `A_i_rhs_no_gauge_terms.C` file**
    1. [Step 1.a](#rhs_from_adirn): *Determining the appropriate RHS based on $A_{i}$*
    1. [Step 1.b](#vi_bi_staggering): *Handling the staggering of $v^{i}$ and $B^{i}$*
    1. [Step 1.c](#staggered_hll): *Computing the generalized HLL formula for $A_{i}$*
1. [Step 2](#code_validation): **Code validation**
1. [Step 3](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0b: Load up cmdline_helper and create the directory
import cmdline_helper as cmd
outdir = os.path.join("..","src")
cmd.mkdir(outdir)

<a id='a_i_rhs_no_gauge_terms__c'></a>

# Step 1: The `A_i_rhs_no_gauge_terms.C` file \[Back to [top](#toc)\]
$$\label{a_i_rhs_no_gauge_terms__c}$$

In this file we implement the right-hand side (RHS) of the evolution equation of the vector potentials $A_{i}$, i.e. the [induction equation](https://en.wikipedia.org/wiki/Induction_equation), excluding gauge terms (see also equation (12) in [Etienne *et al.*](https://arxiv.org/pdf/1501.07276.pdf)). In other words, we implement:

$$
\partial_{t}A_{i}^{\rm NG} \equiv \partial_{t}A_{i} - \left[-\partial_{i}\left(\alpha\Phi - \beta^{j}A_{j}\right)\right]= \partial_{t}A_{i} - \left[\text{gauge terms}\right]_{i} = \psi^{6}\epsilon_{ijk}v^{j}B^{k}\ .
$$

where $\epsilon_{ijk}$ is the [Levi-Civita symbol](https://en.wikipedia.org/wiki/Levi-Civita_symbol) with $\epsilon_{123} = \epsilon_{xyz}=1$, odd permutations of the indices resulting in $-1$, even permutations in $+1$, and if two or more indices are identical in $0$. In components, then, we have

$$
\boxed{
\begin{align}
\partial_{t}A_{x}^{\rm NG} &= \psi^{6}\left(v^{y}B^{z} - v^{z}B^{y}\right) \\
\partial_{t}A_{y}^{\rm NG} &= \psi^{6}\left(v^{z}B^{x} - v^{x}B^{z}\right) \\
\partial_{t}A_{z}^{\rm NG} &= \psi^{6}\left(v^{x}B^{y} - v^{y}B^{x}\right)
\end{align}
}\ .
$$

<a id='rhs_from_adirn'></a>

## Step 1.a: Determining the appropriate RHS based on $A_{i}$ \[Back to [top](#toc)\]
$$\label{rhs_from_adirn}$$

One of the inputs of the function is the variable `A_dirn`, which we will denote by $a=1,2,3$, which is an integer that specifies the direction of $A_{i}$ we are interested in. Now, in order to properly determine the RHS we want to use, we need to remember the following:

* The $v^{i}$ arrays store $\left(v^{x},v^{y},v^{z}\right)$ at consecutive indices, i.e. $\left(\ldots,n-1,\underbrace{n,n+1,n+2}_{\text{indices corresponding to } v^{i}},n+3,\ldots\right)$ ;
* The $B^{i}$ arrays store $\left(B^{x},B^{y},B^{z}\right)$ at consecutive indices, i.e. $\left(\ldots,m-1,\underbrace{m,m+1,m+2}_{\text{indices corresponding to } B^{i}},m+3,\ldots\right)$ .

Consider, then, the quantities:

\begin{align}
O_{1} &\equiv {\rm mod}\left[(a-1)+1,3\right]\ ,\\
O_{2} &\equiv {\rm mod}\left[(a-1)+2,3\right]\ ,
\end{align}

where $O$ stands for *offset*. Let $V$ and $B$ correspond to the $v^{i}$ and $B^{i}$ arrays, respectively, with $V(n)\to v^{x}$ and $B(m)\to B^{x}$. Then, we have the following table:

| $a$ | $A_{a}$ | $O_{1}$ | $O_{2}$ | $V[n+O_{1}]$ | $V[n+O_{2}]$ | $B[m+O_{1}]$ | $B[m+O_{2}]$ |
|:---:|:-------:|:-------:|:-------:|:------------:|:------------:|:------------:|:------------:|
|  1  | $A_{x}$ |    1    |    2    | $\to v^{y}$  | $\to v^{z}$  | $\to B^{y}$  | $\to B^{z}$  |
|  2  | $A_{y}$ |    2    |    0    | $\to v^{z}$  | $\to v^{x}$  | $\to B^{z}$  | $\to B^{x}$  |
|  3  | $A_{z}$ |    0    |    1    | $\to v^{x}$  | $\to v^{y}$  | $\to B^{x}$  | $\to B^{y}$  |

Thus, we can write, in general

$$
\boxed{\partial_{t}A_{a}^{\rm NG} = \psi^{6}V[n+O_{1}]B[m+O_{2}] - V[n+O_{2}]B[m+O_{1}]}\ .
$$

In this first step, we determine the values of $O_{1}$ and $O_{2}$, which are referred to in the code by `v1_offset` and `v2_offset`, respectively. We then also set variables $v_{1}$, $v_{2}$, $B_{1}$, $B_{2}$ which point to the array elements $V[n+O_{1}]$, $V[n+O_{2}]$, $B[m+O_{1}]$, $B[m+O_{2}]$, respectively. Notice that because we have $v^{i}$ and $B^{i}$ on different staggered points, we will set more than one pointer at this stage.

In [2]:
%%writefile $outdir/A_i_rhs_no_gauge_terms.C
/* Compute the part of A_i_rhs that excludes the gauge terms. I.e., we set
 *   A_i_rhs = \partial_t A_i = \psi^{6} (v^z B^x - v^x B^z)   here.
 */
static void A_i_rhs_no_gauge_terms(const int A_dirn,const cGH *cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,gf_and_gz_struct *OUT_PRIMS_R,gf_and_gz_struct *OUT_PRIMS_L,
                                   CCTK_REAL *phi_interped,CCTK_REAL *cmax_1,CCTK_REAL *cmin_1,CCTK_REAL *cmax_2,CCTK_REAL *cmin_2, CCTK_REAL *A3_rhs) {
  // If A_dirn=1, then v1_offset=1 (v1=VY) and v2_offset=2 (v2=VZ)
  // If A_dirn=2, then v1_offset=2 (v1=VZ) and v2_offset=0 (v2=VX)
  // If A_dirn=3, then v1_offset=0 (v1=VX) and v2_offset=1 (v2=VY)
  int v1_offset = ((A_dirn-1)+1)%3,           v2_offset = ((A_dirn-1)+2)%3;

  CCTK_REAL *v1rr=OUT_PRIMS_R[VXR+v1_offset].gf, *v2rr=OUT_PRIMS_R[VXR+v2_offset].gf;
  CCTK_REAL *v1rl=OUT_PRIMS_L[VXR+v1_offset].gf, *v2rl=OUT_PRIMS_L[VXR+v2_offset].gf;
  CCTK_REAL *v1lr=OUT_PRIMS_R[VXL+v1_offset].gf, *v2lr=OUT_PRIMS_R[VXL+v2_offset].gf;
  CCTK_REAL *v1ll=OUT_PRIMS_L[VXL+v1_offset].gf, *v2ll=OUT_PRIMS_L[VXL+v2_offset].gf;

  CCTK_REAL *B1r=OUT_PRIMS_R[BX_STAGGER+v1_offset].gf, *B1l=OUT_PRIMS_L[BX_STAGGER+v1_offset].gf;
  CCTK_REAL *B2r=OUT_PRIMS_R[BX_STAGGER+v2_offset].gf, *B2l=OUT_PRIMS_L[BX_STAGGER+v2_offset].gf;

Overwriting ../src/A_i_rhs_no_gauge_terms.C


<a id='vi_bi_staggering'></a>

## Step 1.b: Handling the staggering of $v^{i}$ and $B^{i}$ \[Back to [top](#toc)\]
$$\label{vi_bi_staggering}$$

Now that we have the correct $v^{i}$ and $B^{i}$ component, we must be careful about how these quantities are staggered with respect to $A_{i}$. Remember that the staggering of these functions is the following:

|    $a$    |          $A_{a}$           |   $v^{n+O_{1},n+O_{2}}$    |       $B^{m+O_{1}}$        |       $B^{m+O_{2}}$       |
|:---------:|:--------------------------:|:--------------------------:|:--------------------------:|:--------------------------:|
|     x     |$\left(i,j+\frac{1}{2},k+\frac{1}{2}\right)$|$\left(i,j-\frac{1}{2},k-\frac{1}{2}\right)$|$\left(i,j+\frac{1}{2},k-\frac{1}{2}\right)$|$\left(i,j-\frac{1}{2},k+\frac{1}{2}\right)$|
|     y     |$\left(i+\frac{1}{2},j,k+\frac{1}{2}\right)$|$\left(i-\frac{1}{2},j,k-\frac{1}{2}\right)$|$\left(i-\frac{1}{2},j,k+\frac{1}{2}\right)$|$\left(i+\frac{1}{2},j,k-\frac{1}{2}\right)$|
|     z     |$\left(i+\frac{1}{2},j+\frac{1}{2},k\right)$|$\left(i-\frac{1}{2},j-\frac{1}{2},k\right)$|$\left(i+\frac{1}{2},j-\frac{1}{2},k\right)$|$\left(i-\frac{1}{2},j+\frac{1}{2},k\right)$|

Now, obtaining, for example, $v^{n+O_{1},n+O_{2}}=v^{y,z}$ (first case in the table above) at the correct point, i.e. $\left(i,j+\frac{1}{2},k+\frac{1}{2}\right)$, would correspond to shifting the gridpoints of the $v^{y,z}$ gridfunctions up by $1$ in the $y,z$-directions. Similarly, obtaining $B^{m+O_{1}}=B^{y}$ at the point $\left(i,j+\frac{1}{2},k+\frac{1}{2}\right)$ corresponds shifting its index up by $1$ in the $z$-direction, while obtaining $B^{m+O_{2}}=B^{z}$ at the point $\left(i,j+\frac{1}{2},k+\frac{1}{2}\right)$ corresponds shifting its index up by $1$ in the $y$-direction.

Below, we set all these offsets by defining the `vs_ijk_offset`, `B1_ijk_offset`, and `B2_ijk_offset` arrays, respectively.

In [3]:
%%writefile -a $outdir/A_i_rhs_no_gauge_terms.C


  /**** V DEPENDENCIES ****/
  /* In the case of Ax_rhs, we need v{y,z}{r,l} at (i,j+1/2,k+1/2). 
   *    However, v{y,z}{r,l}{r,l} are defined at (i,j-1/2,k-1/2), so
   *    v{y,z}{r,l} at (i,j+1/2,k+1/2) is stored at v{y,z}{r,l}{r,l}(i,j+1,k+1).
   * In the case of Ay_rhs, we need v{x,z}{r,l} at (i+1/2,j,k+1/2). 
   *    However, v{x,z}{r,l}{r,l} are defined at (i-1/2,j,k-1/2), so
   *    v{x,z}{r,l} at (i+1/2,j,k+1/2) is stored at v{x,z}{r,l}{r,l}(i+1,j,k+1).
   * In the case of Az_rhs, we need v{x,y}{r,l} at (i+1/2,j+1/2,k). 
   *    However, v{x,y}{r,l}{r,l} are defined at (i-1/2,j-1/2,k), so
   *    v{x,y}{r,l} at (i+1/2,j+1/2,k) is stored at v{x,y}{r,l}{r,l}(i+1,j+1,k). */
  static const int vs_ijk_offset[4][3] = { {0,0,0} , {0,1,1} , {1,0,1} , {1,1,0} };

  /**** B DEPENDENCIES ****/
  /* In the case of Ax_rhs, we need B{y,z}{r,l} at (i,j+1/2,k+1/2).
   *    However, By_stagger{r,l} is defined at (i,j+1/2,k-1/2), and 
   *             Bz_stagger{r,l} is defined at (i,j-1/2,k+1/2), so
   *             By_stagger{r,l} at (i,j+1/2,k+1/2) is stored at By_stagger{r,l}(i,j,k+1), and
   *             Bz_stagger{r,l} at (i,j+1/2,k+1/2) is stored at Bz_stagger{r,l}(i,j+1,k).
   * In the case of Ay_rhs, we need B{z,x}_stagger{r,l} at (i+1/2,j,k+1/2).
   *    However, Bz_stagger{r,l} is defined at (i-1/2,j,k+1/2), and
   *             Bx_stagger{r,l} is defined at (i+1/2,j,k-1/2), so
   *             Bz_stagger{r,l} at (i+1/2,j,k+1/2) is stored at Bz_stagger{r,l}(i+1,j,k), and
   *             Bx_stagger{r,l} at (i+1/2,j,k+1/2) is stored at Bx_stagger{r,l}(i,j,k+1).
   * In the case of Az_rhs, we need B{x,y}_stagger{r,l} at (i+1/2,j+1/2,k).
   *    However, Bx_stagger{r,l} is defined at (i+1/2,j-1/2,k), and 
   *             By_stagger{r,l} is defined at (i-1/2,j+1/2,k), so
   *             Bx_stagger{r,l} at (i+1/2,j+1/2,k) is stored at Bx_stagger{r,l}(i,j+1,k), and
   *             By_stagger{r,l} at (i+1/2,j+1/2,k) is stored at By_stagger{r,l}(i+1,j,k).
   */
  static const int B1_ijk_offset[4][3] = { {0,0,0} , {0,0,1} , {1,0,0} , {0,1,0} };
  static const int B2_ijk_offset[4][3] = { {0,0,0} , {0,1,0} , {0,0,1} , {1,0,0} };

Appending to ../src/A_i_rhs_no_gauge_terms.C


<a id='staggered_hll'></a>

## Step 1.c: Computing the generalized HLL formula for $A_{i}$ \[Back to [top](#toc)\]
$$\label{staggered_hll}$$

We now define the following quantities:

$$
\left(
B^{m+O_{1}}_{L},
B^{m+O_{1}}_{R},
B^{m+O_{2}}_{L},
B^{m+O_{2}}_{R}
\right)\ .
$$

Then, define $\left(\mathcal{E}_{a}\right)_{AB} \equiv \left(\partial_{t}A_{a}^{\rm NG}\right)_{AB}$, where $A$ and $B$ can be $L$ or $R$ (for right and left face values), which allow us to compute

$$
\begin{align}
\left(\mathcal{E}_a\right)_{RR} &= \psi^{6}\left(v^{n+O_{1}}_{R}B^{m+O_{2}}_{R} - v^{n+O_{2}}_{R}B^{m+O_{1}}_{R}\right)\ .\\
\left(\mathcal{E}_a\right)_{RL} &= \psi^{6}\left(v^{n+O_{1}}_{R}B^{m+O_{2}}_{R} - v^{n+O_{2}}_{R}B^{m+O_{1}}_{L}\right)\ ,\\
\left(\mathcal{E}_a\right)_{LR} &= \psi^{6}\left(v^{n+O_{1}}_{L}B^{m+O_{2}}_{L} - v^{n+O_{2}}_{L}B^{m+O_{1}}_{R}\right)\ ,\\
\left(\mathcal{E}_a\right)_{LL} &= \psi^{6}\left(v^{n+O_{1}}_{L}B^{m+O_{2}}_{L} - v^{n+O_{2}}_{L}B^{m+O_{1}}_{L}\right)\ .
\end{align}
$$

Then, having also set the characteristic speeds $c^{\pm}_{m + O_{1}}$ and $c^{\pm}_{m + O_{2}}$, we compute the standard HLL formula generalized to staggered gridfunctions (see equation (33) in [Etienne *et al.*](https://arxiv.org/pdf/1501.07276.pdf))

$$
\boxed{
\begin{align}
\left(\mathcal{E}_a\right)^{\rm HLL}
&=
\frac{
c^{+}_{m+O_{1}}c^{+}_{m+O_{2}}\left(\mathcal{E}_a\right)_{LL} +
c^{+}_{m+O_{1}}c^{-}_{m+O_{2}}\left(\mathcal{E}_a\right)_{LR} +
c^{-}_{m+O_{1}}c^{+}_{m+O_{2}}\left(\mathcal{E}_a\right)_{RL} +
c^{-}_{m+O_{1}}c^{-}_{m+O_{2}}\left(\mathcal{E}_a\right)_{RR}
}
{\left(c^{+}_{m+O_{1}}+c^{-}_{m+O_{1}}\right)\left(c^{+}_{m+O_{2}}+c^{-}_{m+O_{2}}\right)}\\
&+
\frac{c^{+}_{m+O_{1}}c^{-}_{m+O_{1}}}{c^{+}_{m+O_{1}}+c^{-}_{m+O_{1}}}\left(B^{m+O_{2}}_{R}-B^{m+O_{2}}_{L}\right)
+
\frac{c^{+}_{m+O_{2}}c^{-}_{m+O_{2}}}{c^{+}_{m+O_{2}}+c^{-}_{m+O_{2}}}\left(B^{m+O_{1}}_{R}-B^{m+O_{1}}_{L}\right)
\end{align}
}\ .
$$

As a more practical example, which has a far less cluttered notation, consider the formula above for $a=z$. We then have

$$
\begin{align}
\left(\mathcal{E}_z\right)^{\rm HLL}
&=
\frac{
c^{+}_{x}c^{+}_{y}\left(\mathcal{E}_z\right)_{LL} +
c^{+}_{x}c^{-}_{y}\left(\mathcal{E}_z\right)_{LR} +
c^{-}_{x}c^{+}_{y}\left(\mathcal{E}_z\right)_{RL} +
c^{-}_{x}c^{-}_{y}\left(\mathcal{E}_z\right)_{RR}
}
{\left(c^{+}_{x}+c^{-}_{x}\right)\left(c^{+}_{y}+c^{-}_{y}\right)}\\
&+
\frac{c^{+}_{x}c^{-}_{x}}{c^{+}_{x}+c^{-}_{x}}\left(B^{y}_{R}-B^{y}_{L}\right)
+
\frac{c^{+}_{y}c^{-}_{y}}{c^{+}_{y}-c^{-}_{y}}\left(B^{x}_{R}-B^{x}_{L}\right)\ ,
\end{align}
$$

which is precisely equation (33) in [Etienne *et al.*](https://arxiv.org/pdf/1501.07276.pdf).

In [4]:
%%writefile -a $outdir/A_i_rhs_no_gauge_terms.C


#pragma omp parallel for
  for(int k=cctk_nghostzones[2];k<cctk_lsh[2]-cctk_nghostzones[2];k++) for(int j=cctk_nghostzones[1];j<cctk_lsh[1]-cctk_nghostzones[1];j++) for(int i=cctk_nghostzones[0];i<cctk_lsh[0]-cctk_nghostzones[0];i++) {
        int index=CCTK_GFINDEX3D(cctkGH,i,j,k);
        // The following lines set the indices appropriately. See justification in exorbitant comments above.
        int index_v =CCTK_GFINDEX3D(cctkGH,i+vs_ijk_offset[A_dirn][0],j+vs_ijk_offset[A_dirn][1],k+vs_ijk_offset[A_dirn][2]);
        int index_B1=CCTK_GFINDEX3D(cctkGH,i+B1_ijk_offset[A_dirn][0],j+B1_ijk_offset[A_dirn][1],k+B1_ijk_offset[A_dirn][2]);
        int index_B2=CCTK_GFINDEX3D(cctkGH,i+B2_ijk_offset[A_dirn][0],j+B2_ijk_offset[A_dirn][1],k+B2_ijk_offset[A_dirn][2]);

        // Stores 1/sqrt(gamma)==exp(6 phi) at (i+1/2,j+1/2,k) for Az, (i+1/2,j,k+1/2) for Ay, and (i,j+1/2,k+1/2) for Az.
        CCTK_REAL psi6_interped=exp(6.0*(phi_interped[index]));

        CCTK_REAL B1lL = B1l[index_B1];
        CCTK_REAL B1rL = B1r[index_B1];
        CCTK_REAL B2lL = B2l[index_B2];
        CCTK_REAL B2rL = B2r[index_B2];

        CCTK_REAL A3_rhs_rr = psi6_interped*(v1rr[index_v]*B2rL - v2rr[index_v]*B1rL);
        CCTK_REAL A3_rhs_rl = psi6_interped*(v1rl[index_v]*B2rL - v2rl[index_v]*B1lL);
        CCTK_REAL A3_rhs_lr = psi6_interped*(v1lr[index_v]*B2lL - v2lr[index_v]*B1rL);
        CCTK_REAL A3_rhs_ll = psi6_interped*(v1ll[index_v]*B2lL - v2ll[index_v]*B1lL);


        // All variables for the A_i_rhs computation are now at the appropriate staggered point,
        //   so it's time to compute the HLL flux!

        // Note that with PPM, cmin and cmax are defined between ijk=3 and ijk<cctk_lsh[]-2 for all directions.
        CCTK_REAL cmax_1L = cmax_1[index_B2];
        CCTK_REAL cmin_1L = cmin_1[index_B2];
        CCTK_REAL cmax_2L = cmax_2[index_B1];
        CCTK_REAL cmin_2L = cmin_2[index_B1];

        CCTK_REAL B1tilder_minus_B1tildel = psi6_interped*( B1rL - B1lL );
        CCTK_REAL B2tilder_minus_B2tildel = psi6_interped*( B2rL - B2lL );

        /*---------------------------
         * Implement 2D HLL flux 
         * [see Del Zanna, Bucciantini & Londrillo A&A 400, 397 (2003), Eq. (44)]
         *
         * Note that cmax/cmin (\alpha^{\pm}  as defined in that paper) is at a slightly DIFFERENT 
         * point than that described in the Del Zanna et al paper (e.g., (i+1/2,j,k) instead of
         * (i+1/2,j+1/2,k) for F3).  Yuk Tung Liu discussed this point with M. Shibata,
         * who found that the effect is negligible.
         ---------------------------*/
        A3_rhs[index] = (cmax_1L*cmax_2L*A3_rhs_ll + cmax_1L*cmin_2L*A3_rhs_lr +
                         cmin_1L*cmax_2L*A3_rhs_rl + cmin_1L*cmin_2L*A3_rhs_rr)
          /( (cmax_1L+cmin_1L)*(cmax_2L+cmin_2L) ) 
          - cmax_1L*cmin_1L*(B2tilder_minus_B2tildel)/(cmax_1L+cmin_1L) 
          + cmax_2L*cmin_2L*(B1tilder_minus_B1tildel)/(cmax_2L+cmin_2L);
      }
}



Appending to ../src/A_i_rhs_no_gauge_terms.C


<a id='code_validation'></a>

# Step 2: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [5]:
# # Verify if the code generated by this tutorial module
# # matches the original IllinoisGRMHD source code

# # First download the original IllinoisGRMHD source code
# import urllib
# from os import path

# original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/A_i_rhs_no_gauge_terms.C"
# original_IGM_file_name = "A_i_rhs_no_gauge_terms-original.C"
# original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# # Then download the original IllinoisGRMHD source code
# # We try it here in a couple of ways in an attempt to keep
# # the code more portable
# try:
#     original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
#     # Write down the file the original IllinoisGRMHD source code
#     with open(original_IGM_file_path,"w") as file:
#         file.write(original_IGM_file_code)
# except:
#     try:
#         original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
#         # Write down the file the original IllinoisGRMHD source code
#         with open(original_IGM_file_path,"w") as file:
#             file.write(original_IGM_file_code)
#     except:
#         # If all else fails, hope wget does the job
#         !wget -O $original_IGM_file_path $original_IGM_file_url

# # Perform validation
# Validation__A_i_rhs_no_gauge_terms__C  = !diff $original_IGM_file_path $outfile_path__A_i_rhs_no_gauge_terms__C

# if Validation__A_i_rhs_no_gauge_terms__C == []:
#     # If the validation passes, we do not need to store the original IGM source code file
#     !rm $original_IGM_file_path
#     print("Validation test for A_i_rhs_no_gauge_terms.C: PASSED!")
# else:
#     # If the validation fails, we keep the original IGM source code file
#     print("Validation test for A_i_rhs_no_gauge_terms.C: FAILED!")
#     # We also print out the difference between the code generated
#     # in this tutorial module and the original IGM source code
#     print("Diff:")
#     for diff_line in Validation__A_i_rhs_no_gauge_terms__C:
#         print(diff_line)

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.pdf](Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [6]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.tex
!rm -f Tut*.out Tut*.aux Tut*.log